In [ ]:
import os
import sys


### 容器类型
###  len   contains  getitem  setitem


In [ ]:
l = [1, 2, 3, 4, 5]
# 1 in l  #  这个的本质是在类中实现了了 __contains__() 魔法方法
class Foo(object):
    def __init__(self):
        self.x = 123
        self.y = 456
        print('Foo constructor')
        
    def __contains__(self, value):
        return self.x == value or self.y == value
     
foo = Foo() 
123 in foo 

### 取索引操作 list tuple str bytes set(不支持) dict

In [ ]:
# l[0] 本质就是__getitem__()
l.__getitem__(1)
# dict 特殊，取的是key
# list dict 支持修改 所以可以使用__setitem__()

### contextmanguage  __enter__() __exit__()   with语句

In [ ]:
import traceback

errors = []
class myabc(object):
    def __enter__(self):
        print('I am enter')
        return self  #添加了 return self 的效果 = <__main__.myabc object at 0x7faeac44a6a0> 也就是返回对象
        
    def close(self):
        print('context close')
        
    def __exit__(self, exc_type, exc_val, exc_tb): #traceback exc_type对应异常类型，exc_value是异常值（ValueError的实例），exc_tb是那个对象发生异常。
        print('I am exit')
        print(f'exc_type={exc_type} exc_val={exc_val} exc_tb={exc_tb}')
        errors.extend([exc_type, exc_val, exc_tb])
        # self.close()  # 文件操作在这里执行这句  fp.__enter__() fp.__exit__()  fp.closed() 以关闭(文件状态打开关闭)
        # lck = Lock() 最好用 with Lock() as lck  acquire release



In [ ]:
with myabc() as abc:   # 不要放在一个cell 里。
    print(abc)
    raise ValueError('手动抛出异常') # 加了这句__exit__()函数中的三个值以不返回None,而是有内容了。



In [ ]:
a, b, c = errors  # 不要放在一个cell 里。
print(a) 
print(b)
print(c)
traceback.print_exception?
traceback.print_exception(a, b, c)

In [ ]:
setattr(abc, 'china', 'I like china')
print(abc.china)
res_str = getattr(abc, "china")
print(res_str)
bool_res = hasattr(abc, "china")
print(bool_res)

In [ ]:
abc.city = 'shanghai' # 添加对象属性的本质是调用了类中__setattr__() 我们自己写的类中可以覆盖父类中的。
# __getattribute__() 比如abc.city(底层操作)  (没有__getattr__()） __dict__()



In [ ]:
class Test:
    l = [6, 7, 8]
    
    def __init__(self):
        self.mony = 10000
        self.x = 666
        self.y = 'china'
        print('Test constructor')
        
    def __setattr__(self, name, value):  # 用于类型检查,判断（监听属性的变化）相当与(HOOK)。
        if name == 'money' and value < 0:
            raise ValueError('money: %s < 0 ' % value)
            
        print(f'__setattr__ name={name}, value={value}')
        object.__setattr__(self, name, value)
    
    def __getattribute__(self, name):
        print(f'getattribute name={name}')
        return object.__getattribute__(self, name)  # 在本机ipython 运行正常，在线不知道为什么错误。(!!!!!!!!!使用 %reset 清空正常)
    
    def __getattr__(self, name):
        '''处理属性和方法不存在时的默认行为'''
        print("__getattr__= %s" % (name))
        return -1
    
    def __delattr__(self, name):
        if name == 'age':
            print('delete...%s' % name)
            object.__delattr__(self, name)
    
    def myadd(self, x, y):
        return x + y

In [ ]:
test = Test() # 调用__init__(self) 打印出来ipy


In [ ]:
test.x -= 10 # 本质是调用Test类的__setattr__(self)  等价与= test.x = test.x - 10 先取了一次 在赋值 共2次 __getattribute__ __setattr__
#print(test.__dict__)
test.y  # 本质是调用Test类的__getattribute__(self,kyename)
Test.l # 获取类属性l 也是通过 __getattribute__
test.myadd(3, 5) #  调用类的实例属性也调用了 __getattribut__() 
# 所有实例对象点的操作都调用了 __getattribute__()
# __getattr__() 的触发。
test.z # 调用一个不存在的属性触发。__getattr__()  如果类中没是实现魔法方法__getattr__() 就会抛出异常。

### 开发中使用__getattr__() 方法，有点向HOOK 钩子。

In [ ]:
# 类 __slots__() = [] or () 把类的属性确定下来 属性字典也给删除了 __dict__[] = '123' 也不行了 内存节省，对象内存分配，加快了__new__构造。